In [25]:
import numpy as np
from itertools import permutations
from collections import OrderedDict
from itertools import islice
import pandas as pd

In [26]:
#Task : [machine: processing time]

job1={1:7, 3:8, 2:10}
job2={2:6, 1:4, 3:12}
job3={1:8, 2:8, 3:7}

jobs=[job1, job2 , job3]

In [27]:
import math
math.pow(3,16)/math.factorial(16)

2.0574082725310405e-06

In [28]:
#maximum machine total time (MTT) is lower bound to makespan. Implement Bound and Branch algorithm in descending MTT order
tt=[0,0,0]

t=[]

for i in range(len(jobs)):
    for j in range(len(jobs)):
        t.append(jobs[i][j+1])
for i in range(len(jobs)):
    tt[0]= tt[0]+t[0+3*i]
    tt[1]= tt[1]+t[1+3*i]
    tt[2]= tt[2]+t[2+3*i]
mtt= max(tt)

def BottleneckMachineOrder():
    return list(reversed([x+1 for x in np.argsort(tt).tolist()]))

In [29]:
#Branch and Bound algorithm for 1 | rj | Lmax problem ( 1 machine, release times for all jobs are known, minimize maximum lateness)
def BranchAndBound(machine, LB):
    
    def ReleaseTime(job, machine):
        o = OrderedDict(jobs[job])
        slicepos=list(o.keys()).index(machine)
        sliced = islice(o.items(), slicepos)  # o.iteritems() is o.items() in Python 3
        sliced = OrderedDict(sliced)
        return (sum(sliced.values()))
    
    def DueDate(job, machine,lb):
        o = OrderedDict(jobs[job])
        slicepos=list(o.keys()).index(machine)
        sliced = islice(o.items(), slicepos+1,None)  # o.iteritems() is o.items() in Python 3
        sliced = OrderedDict(sliced)
        return (lb-sum(sliced.values()))
    
    # 1 | rj | Lmax data : processing time (pj), release time (rj), due date (dj)
    def Data(machine, LB):
        #lb = max(mtt)
        data={'pj':[jobs[0][machine],jobs[1][machine],jobs[2][machine]], 'rj':[ReleaseTime(0,machine),ReleaseTime(1,machine),ReleaseTime(2,machine)],'dj':[DueDate(0, machine,LB),DueDate(1, machine,LB),DueDate(2, machine,LB)]}
        return data
    
    def CompletedTime(problem, order):
        result=[]
        o1=order[0]-1
        o2=order[1]-1
        o3=order[2]-1
        delay = 0

        ct1=problem['rj'][o1]+problem['pj'][o1]
        result.append(ct1)

        if (problem['rj'][o2]<= ct1):
            ct2=ct1+problem['pj'][o2]
        else: 
            ct2=problem['rj'][o2]+problem['pj'][o2]
            delay= problem['rj'][o2]-ct1 #Using Preemptive EDD Rule
        result.append(ct2)

        if (problem['rj'][o3]<= ct2):
            ct3=ct2+problem['pj'][o3]-delay
        else: 
            ct3=problem['rj'][o3]+problem['pj'][o3]-delay
        result.append(ct3)

        return result
    
    def Lmax(problem, order):
        L=[]
        o1=order[0]-1
        o2=order[1]-1
        o3=order[2]-1
        print ("J  ",order)
        DD=[]
        for i in [o1,o2, o3]:
            DD.append(problem['dj'][i])
        for i in range(len(order)):
            L.append(CompletedTime(problem, order)[i] - DD[i] if CompletedTime(problem, order)[i] - DD[i]>0 else 0)
        print("CT ",CompletedTime(problem, order))
        print ("DD ",DD)
        print("----------------")
        print ("L  ",L)
        print ("Lmax= ",max(L))
        print("\n")
        return max(L)
    
    def EDD(problem, first):
        #Choose order using Earliest Due Date rule
        DD=problem['dj']
        order=[]
        for x in np.argsort(DD).tolist():
            order.append(x+1)
        order.remove(first)
        order.insert(0,first)
        return order
    
    
    problem=Data(machine,LB)
    
    print("1 | rj | Lmax problem for Machine ", machine)
    print("Data: ", problem)
    
    print("\n")
    
    L=[]
    order=[]
    for i in range(len(problem['pj'])):
        Li=Lmax(problem, EDD(problem, i+1))
        L.append(Li)
        if Li==0 :
            order=EDD(problem, i+1)
            return order
    branch=[x+1 for x in np.argsort(L).tolist()][0]
    a=[1, 2, 3]
    a.remove(branch)
    perm1=[]
    perm2=[]
    for p in permutations(a):
        perm1.append(p[0])
        perm2.append(p[1])
    order=[branch,perm1[0],perm1[1] ] if Lmax(problem,[branch,perm1[0],perm1[1] ])<=Lmax(problem,[branch,perm2[0],perm2[1] ]) else [branch,perm2[0],perm2[1] ]
    return order

In [30]:
def Makespan (job_data, machine_orders, print_schedule):
    data_raw = []
    for i in range(1, len(job_data)+1):
        for j in job_data[i-1]:
            data_raw = data_raw + [[i,j,job_data[i-1][j]]]
    
    data = pd.DataFrame(data_raw, columns = ['job', 'machine', 'time'])
    data.index += 1
    
    jobs = len(job_data)
    machines = 3
    nodes = len(data_raw)
    
    machine_orders = [[]] + machine_orders
    
    graph = ['s'] + list(range(1,nodes+1))
    graph = dict.fromkeys(graph)
    for i in graph:
        graph[i] = set()
        
    chains = []
    for j in range(1,jobs+1):
        chains_temp = ['s'] + (data.loc[data['job'] == j].index.values.tolist()) + ['f']
        chains.append(chains_temp)
    for m in range(1,machines+1):
        chains_temp = []
        for j in machine_orders[m]:
            chains_temp.append(data.loc[(data['job'] == j) & (data['machine'] == m)].index.values[0])
        chains.append(chains_temp)
        
    for i in range(len(chains)):
        for k in range(len(chains[i])-1):
            graph[chains[i][k]].add(chains[i][k+1])   
    
    for i in graph:
        graph[i] = list(graph[i])    
        
        
    nv = ['s'] + list(range(1,nodes+1)) + ['f']
    nv = dict.fromkeys (nv)
    for i in nv:
        if (i == 's' or i == 'f'):
            nv[i] = 0
        else:
            nv[i] = data.loc[data.index == i]['time'].values[0]
        
    max_time = ['s'] + list(range(1,nodes+1)) + ['f']
    max_time = dict.fromkeys (max_time)
    for i in nv:
        max_time[i] = 0
    
    def no_cycles(listObj):
        return len(listObj) == len(set(listObj))
    
    def path_time(listObj):
        t = 0
        for i in listObj:
            t += nv[i]
        return t
    
    paths = [['s']]
    for path in paths:
        if path[-1] != 'f':
            for i in graph[path[-1]]:
                temp_path = path + [i]
                if no_cycles(temp_path):
                    paths.append(temp_path)
                    if path_time(temp_path) > max_time[i]:
                        max_time[i] = path_time(temp_path)              
    
    makespan = max_time['f']
    
    if (print_schedule == True):
        print('Makespan: ', makespan)
        print('\nSchedule: ')
        for i in range(1, nodes+1):
             print('Job', data.loc[i].values[0],'Machine', data.loc[i].values[1],  ':',
                   max_time[i] - nv[i], '-', max_time[i])
         
    return (makespan)

In [31]:
def ShiftingBottleneckHeuristics():
    print("Job1", job1)
    print("Job2", job2)
    print("Job3", job3)
    print("\n")
    
    bottleneckMachineOrder= BottleneckMachineOrder()
    machineOrders= [[],[],[]]
    LB=mtt
    for i in range(len(jobs)):
        machineOrders[len(jobs)-i-1]=BranchAndBound(bottleneckMachineOrder[i], LB)
        if i == 2:
            LB=Makespan(jobs, machineOrders, True)
        else:
            LB=Makespan(jobs, machineOrders, False)
        

In [32]:
ShiftingBottleneckHeuristics()

Job1 {1: 7, 3: 8, 2: 10}
Job2 {2: 6, 1: 4, 3: 12}
Job3 {1: 8, 2: 8, 3: 7}


1 | rj | Lmax problem for Machine  3
Data:  {'pj': [8, 12, 7], 'rj': [7, 10, 16], 'dj': [17, 27, 27]}


J   [1, 2, 3]
CT  [15, 27, 34]
DD  [17, 27, 27]
----------------
L   [0, 0, 7]
Lmax=  7


J   [2, 1, 3]
CT  [22, 30, 37]
DD  [27, 17, 27]
----------------
L   [0, 13, 10]
Lmax=  13


J   [3, 1, 2]
CT  [23, 31, 43]
DD  [27, 17, 27]
----------------
L   [0, 14, 16]
Lmax=  16


J   [1, 2, 3]
CT  [15, 27, 34]
DD  [17, 27, 27]
----------------
L   [0, 0, 7]
Lmax=  7


J   [1, 3, 2]
CT  [15, 23, 34]
DD  [17, 27, 27]
----------------
L   [0, 0, 7]
Lmax=  7


1 | rj | Lmax problem for Machine  2
Data:  {'pj': [10, 6, 8], 'rj': [15, 0, 8], 'dj': [34, 18, 27]}


J   [1, 2, 3]
CT  [25, 31, 39]
DD  [34, 18, 27]
----------------
L   [0, 13, 12]
Lmax=  13


J   [2, 3, 1]
CT  [6, 16, 24]
DD  [18, 27, 34]
----------------
L   [0, 0, 0]
Lmax=  0


1 | rj | Lmax problem for Machine  1
Data:  {'pj': [7, 4, 8], 'rj': [0, 6, 0], 